In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import locale
from tqdm import tqdm
import numpy as np
import math
from numpy import NaN

In [4]:
def elaborate_file(file_name, sensing_material_table_file):
    sensing_table = pd.read_csv(sensing_material_table_file)
    raw_table = pd.read_csv(file_name)
    # todo


In [5]:
sensing_table = pd.read_csv("../sensing_material_appa1.csv")
raw_table = pd.read_csv("../appa1.csv")

In [6]:
# get the timestamp of the sensing material change date
dates_str = sensing_table.columns.to_list()
dates_str.remove("nm")
dates = [pd.to_datetime(x) for x in dates_str]

In [7]:
# change raw_table ts column to timestamp
raw_table.ts = pd.to_datetime(raw_table.ts.str.slice(start = 0, stop = 24), format = "%a %b %d %Y %H:%M:%S")
NUMBER_OF_SENSORS = 8

In [8]:
raw_table = raw_table.set_index("ts")
raw_table = raw_table.resample("H").mean()
raw_table = raw_table.reset_index(level=0)

In [9]:
def get_sensing_mat(index_date, channel):
    return sensing_table[dates_str[index_date]][sensing_table.nm == channel].values[0]

def find_change_date(index_date_reach, channel):
    actual_sens_mat = get_sensing_mat(index_date_reach, channel)
    if  not '*' in actual_sens_mat:
        return dates[index_date_reach]
    else:
        # find the last change
        if get_sensing_mat(index_date_reach -1,channel) in actual_sens_mat:
            # do the same thing on the previus cell
            return find_change_date(index_date_reach - 1, channel)
        else:
            # the sensor changed channel, find the channel and do the same thing
            channel_before = 1
            while channel_before <= NUMBER_OF_SENSORS:
                if get_sensing_mat(index_date_reach -1,channel_before) in actual_sens_mat:
                    break
                channel_before += 1
            else:
                raise Exception(f"error in find previus position of sensor\n actual index {index_date_reach} type {actual_sens_mat}")
            return find_change_date(index_date_reach - 1, channel_before)


In [10]:

columns = ['ts','sensing_,material','signal_res','heater_res','heater_V','T','RH','P','time_cos','time_sin','date_cos','date_sin','Y','age']
new_data = pd.DataFrame(columns=columns)

last_change_date = [dates[0] for _ in range(NUMBER_OF_SENSORS)]
index_date_reached = 0
actual_sensing_material = [get_sensing_mat(0,i) for i in range(1,NUMBER_OF_SENSORS + 1)]

for index, record in tqdm(raw_table.iterrows(), total = raw_table.shape[0]):
    if record.ts.date() < dates[0].date():
        # skip all data before sensor changing(and sensing mat knowing)
        continue

    if index_date_reached != len(dates) and record.ts.date() == dates[index_date_reached+1].date():
        # reached sensing material change day
        index_date_reached += 1
        for channel in range(NUMBER_OF_SENSORS):
            last_change_date[channel] = find_change_date(index_date_reached, channel+1)

        # update the sensing material array
        actual_sensing_material = [get_sensing_mat(index_date_reached,i) for i in range(1,NUMBER_OF_SENSORS + 1)]

    append_ds = pd.DataFrame(columns=columns)
    for channel in range(1,NUMBER_OF_SENSORS + 1):
        if record["Rs"+str(channel)] == np.NaN:
            continue
        # for every channel add a row
        append_ds.loc[len(append_ds.index)] = [
            record['ts'],
            actual_sensing_material[channel-1],
            record['Rs'+str(channel)],
            record['Rh'+str(channel)],
            record['Vh'+str(channel)],
            record['T'],
            record['RH'],
            record['P'],
            math.cos(record['ts'].hour) * 2 * (math.pi / 24),
            math.sin(record['ts'].hour) * 2 * (math.pi / 24),
            math.cos(record['ts'].month) * 2 * (math.pi / 12),
            math.sin(record['ts'].month) * 2 * (math.pi / 12),
            record['ts'].year,
            record['ts'] - last_change_date[channel-1]
        ]
    new_data = new_data.append(append_ds)


  0%|          | 0/25534 [00:00<?, ?it/s]/tmp/ipykernel_4438/895334185.py:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if record.ts.date() < pd.Timestamp(dates[0]):
 91%|█████████ | 23192/25534 [10:19<01:02, 37.43it/s]  


IndexError: list index out of range

In [ ]:
print(new_data)
new_data.to_csv("../appa1_new.csv", index = False)